# List Preview or Live pages on EDS

* Get the authentication token from browser
* Specify "preview" or "live" pages to be listed

In [ ]:
// Add your authentication token here
const token = 'for admin.hlx.page';
// Update the content files to be listeed to "preview" or "live"
const content = 'live';
// Update the repo to "dc" or "da-dc"
const repo = 'da-dc';

async function logError(response) {
  console.error(`Request failed with status: ${response.status} ${response.statusText}`);
  
  console.error('Response Headers:');
  for (const [key, value] of response.headers.entries()) {
    console.error(`  ${key}: ${value}`);
  }
  
  try {
    const errorData = await response.json();
    console.error('Error Response Body:', errorData);
  } catch (e) {
    const errorText = await response.text();
    console.error('Error Response Body (text):', errorText);
  }
  
  throw new Error(`HTTP ${response.status}: ${response.statusText}`);
}

* Test if the token is working

In [ ]:
const response = await fetch(`https://admin.hlx.page/status/adobecom/${repo}/main/`, {
  headers: {
    'x-auth-token': token
  }
});

if (!response.ok) {
  logError(response);
}

const data = await response.json();
console.log(data);

* Create the status job to retrieve the list of pages 

In [ ]:
const response = await fetch(`https://admin.hlx.page/status/adobecom/${repo}/main/*`, {
  method: 'POST',
  headers: {
    'x-auth-token': token,
    'Content-Type': 'application/json'
  },

  body: JSON.stringify({
    "select": [
      content
    ],
    "paths": [
      "/*"
    ],
    "pathsOnly": true
  })
});

if (!response.ok) {
  logError(response);
}

const data = await response.json();
console.log(data);

const jobLink = data.links.self;

* Check the job status until the state is "stopped" 

In [ ]:
const response = await fetch(jobLink, {
  headers: {
    'x-auth-token': token
  }
});

if (!response.ok) {
  logError(response);
}

const data = await response.json();
console.log(data);

const jobDetails = data.links.details;

* Get the job details and save the list of pages

In [ ]:
const response = await fetch(jobDetails, {
  headers: {
    'x-auth-token': token
  }
});

if (!response.ok) {
  logError(response);
}

const data = await response.json();
const fileList = data.data.resources[content];

const fileListString = fileList.join('\n');

console.log(`${content} pages:`);
console.log(fileListString);

const fileName = `${repo}-${content}.txt`;
await Deno.writeTextFile(fileName, fileListString);
console.log(`\nOutput saved to: ${fileName}`);